In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-15 13:06:34--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-15 13:06:34 (76.4 MB/s) - ‘minsearch.py.2’ saved [4073/4073]



In [2]:
import minsearch
import requests 

/workspaces/LLM_zoomcamp/Homework_1/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
q = 'How do execute a command on a Kubernetes pod?'


In [7]:
index.fit(documents)

In [8]:
boost={'question':4,"type": "best_fields"}
results=index.search(
    query=q,
    boost_dict=boost,
    num_results=1
)

In [9]:
results

[{'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'}]

In [11]:
import openai
import os

In [25]:
from openai import OpenAI

In [26]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
)

In [27]:
q = 'the course has already started, can I still enroll?'

In [28]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [29]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [30]:
def llm(prompt):
    response = client.chat.completions.create(
        model='deepseek/deepseek-prover-v2:free',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [31]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [32]:
rag(query)

"### Running Kafka in Java\n\nTo run a Kafka producer, consumer, or KStreams application from the terminal in Java, you can use the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your JAR file. This command sets the classpath to include both the JAR file and the `out` directory, and specifies the main class to run (`JsonProducer.java` in this example).\n\n### Running Kafka in Python\n\n**Setting Up the Environment:**\n\nBefore running Kafka in Python, ensure that you have set up a virtual environment and installed the necessary dependencies. Here are the steps:\n\n1. **Create a Virtual Environment:**\n   ```bash\n   python -m venv env\n   ```\n\n2. **Activate the Virtual Environment:**\n   - On MacOS and Linux:\n     ```bash\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```cmd\n     env\\Scripts\\activate\n     ```\n\n3. **Install Depend